In [1]:
import pandas as pd
# import sqlalchemy
from sqlalchemy import create_engine
import urllib.parse

import configparser
# import os
from pathlib import Path
import numpy as np

import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

from neo4j import GraphDatabase

import datetime
import os
from typing import Dict
import traceback

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
config_file_path = './config.ini'

if not Path(config_file_path).exists():
    logger.warning(f"Config file {config_file_path} not found!")

config = configparser.ConfigParser()
config.read(config_file_path)

if 'DATABASE' not in config:
    raise ValueError("DATABASE section not found in config")

db_config = {
    'host': config['DATABASE']['host'],
    'port': int(config['DATABASE']['port']),
    'username': config['DATABASE']['username'],
    'password': config['DATABASE']['password'],
    'database': config['DATABASE']['database'],
    'query_request': config['DATABASE']['query1'],
    'query_assets': config['DATABASE']['query2'],
    'query_request_with_activities': config['DATABASE']['query3'],
    'schema': config['DATABASE']['schema']
}

db_host = db_config.get('host')
db_port = db_config.get('port')
db_username = db_config.get('username')
db_password = db_config.get('password')
db_database = db_config.get('database')
db_query1 = db_config.get('query_request')
db_query2 = db_config.get('query_assets')
db_query3 = db_config.get('query_request_with_activities')
db_schema = db_config.get('schema')

In [4]:
def database_connector( db_type, host, port, database, username, password, **kwargs):
    
        encoded_password = urllib.parse.quote_plus(password)
        connection_strings = {
            'postgresql': f"postgresql://{username}:{encoded_password}@{host}:{port}/{database}",
        }
        return connection_strings[db_type]

In [5]:
conn_string = database_connector(
            db_type='postgresql',
            host=db_host,
            port=db_port,
            database=db_database,
            username=db_username,
            password=db_password,
            schema=db_schema
            )

In [6]:
last_sync_date_time = '2025-11-01 07:00:00.000'
# '2025-11-01 07:00:02.929'

diff_time = pd.to_datetime(last_sync_date_time) + datetime.timedelta(days=1)
# upper_bound = pd.to_datetime(diff_time, format='mixed').strftime('%Y-%m-%d %H:%M:%S.%f')[:-3]
upper_bound = '2025-11-02 08:00:00.000'


In [7]:
target_dir_path = './data_not_pm'

In [8]:
try:
        engine = create_engine(conn_string)
        
        if not db_query1:
            logger.warning("Query for v_request is missing!")
            
        else:
            db_query1 = db_query1.replace(';',f'\nWHERE "workType" <> \'PM\';')

        if not db_query2:
            logger.warning("Query for v_assets is missing!")
            
        # print(db_query1)
        # print(db_query2)
       
        df_request = pd.read_sql(db_query1, engine)
        df_assets = pd.read_sql(db_query2, engine)
        
        logger.info(f" Downloaded {len(df_request)} rows from 'v_requests', {len(df_assets)} rows from 'v_assets'.")
        
        # # Saving the data
        df_request.to_csv(f"{target_dir_path}/v_requests_not_pm.csv",index=False)
        df_assets.to_csv(f"{target_dir_path}/v_assets.csv",index=False)
        
        logger.info(f"Data exported to {target_dir_path}")
        
except Exception as e:
    logger.warning(f"Error connecting to database: {e}")
    

finally:
    if 'engine' in locals():
            engine.dispose()


SELECT "requestId", "requestAlternateId", "workType", "requestDescription", "requestCreatedDate",
"requestTargetCompletionDate", "requestCompletionDate", "serviceClassificationId", "serviceClassificationAlternateId",
"serviceClassificationPath", "locationId", "locationAlternateId", "locationPath", "assetId", "assetAlternateId",
"assetDescription", "completionNotes", "customer", "country", "isSelfAssign", "priorityCode"
FROM aiml.v_request
WHERE "workType" <> 'PM';
SELECT "Customer", "Country", "Location Alt Id", "Location Id", "Position Alt Id", "Position Id" as "positionId",
"Asset Alt Id", "Asset Id" as "assetId", "Asset Description", "Position Name" as "positionName", "Manufacturer" as "manufacturer",
"Model" as "model", "Serial Number" as "serialNumber"
FROM aiml.v_assets;


INFO:__main__: Downloaded 203544 rows from 'v_requests', 508867 rows from 'v_assets'.
INFO:__main__:Data exported to ./data_not_pm


In [9]:
df_request

,requestId,requestAlternateId,workType,requestDescription,requestCreatedDate,requestTargetCompletionDate,requestCompletionDate,serviceClassificationId,serviceClassificationAlternateId,serviceClassificationPath,...,locationAlternateId,locationPath,assetId,assetAlternateId,assetDescription,completionNotes,customer,country,isSelfAssign,priorityCode
0,000079fd-3b8b-4905-9b3c-a36c04316281,RCHR233869,Reactive,"Room N320 toilet lead is broken, patient in th...",2025-09-18 15:10:41.085,2025-10-09 15:09:33,NaT,a011beda-da97-4bd0-85bf-43df99f5e4b9,SCCHR100097,Plumbing | Fixtures | Toilet (Patient Room/Pub...,...,LUS226145,"US,TX,Texarkana / ALT-T St. Michael Acute Hosp...",None,None,None,None,Christus Health,US,False,P7
1,0000e256-4aa8-46f4-a04b-0b23c853c183,RCHR102814,Proactive,running calls,2025-04-11 13:40:24.231,2025-04-30 18:39:23,2025-04-12 20:11:00.000,dc08b849-48d8-4e49-900a-62a7ffc6991c,SCCHR100258,Interior | Specialty Services | Interior Plant...,...,LUS227292,"US,TX,Texarkana / ALT-T St. Michael Acute Hosp...",None,None,None,ren work orders,Christus Health,US,False,P7
2,00013c96-006b-462b-90f9-58c9aeada7f3,R114045,Reactive,Atrium 1 amenity plantroom -- AHU43 - gas burn...,2025-02-27 12:11:10.232,2025-03-29 13:11:09,2025-02-27 13:26:04.989,c09fd283-a915-48c3-bf4c-a29eae7f9910,SCNEC100437,Maintenance & Repair | Plumbing | Plumbing Rep...,...,LGB112074,National Exhibition Centre / Atrium 1 / First ...,None,None,None,None,NEC,GB,False,P4
3,000156a3-3464-421d-9233-b73ee037e092,R115309,Reactive,Women's Handicap Restroom shower not draining ...,2025-03-18 17:26:12.359,2025-04-01 17:26:11,2025-06-21 02:36:20.773,b9a55196-2833-4374-903c-e656646c5ce8,SCSCE100512,Maintenance & Repair | Plumbing | Plumbing Rep...,...,LUS277190,"US, CA, Long Beach / Long Beach SC - Main Bldg...",None,None,None,None,Southern California Edison,US,False,P4
4,0002845d-7b2c-46f6-a525-42576a787a07,RSDL106407,Reactive,"Buongiorno, richiedo un paio di scarpe modello...",2025-03-20 10:07:40.976,2025-04-19 10:06:40,2025-03-25 13:04:00.000,ebd18617-eb46-434a-aa5c-e0ed9842e15b,SCSDL100254,Office Services | Other Services | Workwear re...,...,LIT102719,"IT, Italy, Parma Factory / Manufactoring - Off...",None,None,None,completato,Sidel,IT,False,P1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203539,ffff1b64-2b08-424b-9862-15503bea9e12,RCLX104453,Reactive,"At the Strickland building, dishwasher on the ...",2025-09-04 21:06:06.418,2025-10-04 21:05:05,2025-09-24 18:23:00.000,5cb3a1e1-d8e3-4365-9ccc-5e69d1f7d047,SCCLX100071,Maintenance & Repair | Cafeteria/Breakroom | V...,...,LUS236664,"US, NC, Durham / Durham / Floor 01 / Worker Bees",None,None,None,work completed,Clorox,US,False,P5
203540,ffff9203-fb10-45a8-b029-fb4f2d0de552,RTHD105185,Corrective,Clean warehouse floor,2025-02-10 11:33:54.447,2025-03-12 11:33:53,2025-02-10 13:37:47.683,777996e5-dbfb-4042-b4b9-8cad1d2cdcc9,SCTHD100170,Janitorial | Cleaning | Floor Clean General,...,LUS181695,"US, GA, Atlanta / Technical Distribution Cente...",None,None,None,Cleaned warehouse,Home Depot Corp,US,False,P7
203541,ffff957d-0a72-4de3-b001-48fed2d79f6b,R110704,Reactive,วันที่ 12 Mar ช่วงบ่าย / ส่งเอกสารมอบอำนาจ ที่...,2025-03-12 03:41:18.630,2025-03-19 03:41:17,2025-03-13 02:54:21.273,0dc19d23-f2da-4890-a3cb-4118bc895fb3,SCSYN100137,Office Services | Mail Room | Deliver/Pick-up ...,...,LTH100324,"TH, BANGKOK / THBK - Bangkok Cyber World Tower...",None,None,None,None,Syngenta,TH,False,P14
203542,ffffc309-571f-4aae-a455-4336b280f598,RABV107470,Proactive,Light out on the third floor at the top of the...,2025-09-04 12:28:13.910,2025-09-29 19:27:12,NaT,a0412648-2179-47a9-908b-45ee0c3f841c,SCABV100239,Maintenance & Repair | Electrical | Electrical...,...,LUS346799,"US, NJ, Madison / MADISON / PARKING P-L2 / All",None,None,None,None,AbbVie,US,False,P4


In [10]:
df_assets

,Customer,Country,Location Alt Id,Location Id,Position Alt Id,positionId,Asset Alt Id,assetId,Asset Description,positionName,manufacturer,model,serialNumber
0,Fifth Third Bank,US,LUS143988,904198e9-9996-4773-af46-cd4f5f0b90d0,AUS208785,734762d2-f609-46d1-8ee8-fa9095f019f5,AUS208832,bf420915-50ca-4999-9388-0e46b94f0194,Teller stools(Teller Stools),Teller Stools,No Manufacturer Data,No Model Data,UNK82505
1,Fifth Third Bank,US,LUS137218,e504a99d-aad0-4322-b7af-b5947eb270b9,AUS167979,72d8c56f-1df1-4090-9319-e3f99934c599,AUS168004,b8566f9d-0407-4542-8bcb-68815b89be7e,Quarterly HVAC Equipment Inspection(Quarterly ...,Quarterly HVAC Equipment Inspection,No Manufacturer Data,No Model Data,UNK21677
2,RJ Reynolds,US,LUS187382,7bd5f0b6-ecf2-43bd-8e87-7a454866141b,AUS424732,c2e3baa9-e031-4988-8425-bb1d51dec863,AUS424994,1e8c3724-864e-434d-8867-1dc843fa1898,Ford Pickup(Ford Truck),Ford PU from Kernsville,Ford,Pickup504197,F1
3,Fifth Third Bank,US,LUS143946,6091189b-a13b-42a8-a33d-6f483942aa19,AUS208475,92107f21-6d4c-44f9-bc34-ce1fdc9257f6,AUS208522,7a644146-e695-43fb-9e0c-edda9df5ef85,Condenser Unit(CU-02),CU-02,TRANE,TWA090D30RAA,142527CMYA
4,Fifth Third Bank,US,LUS144564,b82eb508-fc5c-4a65-aede-49cda32c1663,AUS211284,bcda07a4-afa0-4c26-8ca0-3c2375fd1dc7,AUS211332,032407c1-638b-4247-b16f-0589ec3ab23c,Irrigation System - Unknown # of Zones/Not Ass...,P2020::TN101737.Irrigation System,No Manufacturer Data,No Model Data,UNK211284
...,...,...,...,...,...,...,...,...,...,...,...,...,...
508862,Syngenta,FR,LFR108617,483f0e1f-302d-470a-bc3c-eb476f754db5,AFR112004,fffe241a-32ff-48b3-b4c0-ebb6f88638a4,AFR112004,9e4542b7-09da-4549-927d-72ca7ff3252c,FRSC-FEX-Extincteur n°41 - Poudre 2Kg - BA VEH...,FRSC-FEX-Extincteur n°41 - Poudre 2Kg - BA VEH...,DESAUTEL,PP2P,GYORA
508863,Fifth Third Bank,US,LUS133219,800daba7-f960-4444-9187-c3cf0df56c69,AUS145841,fffe6bc0-6417-4146-be87-854eb79cbf50,AUS145846,14597f25-0baf-4c97-944a-d38f978c5293,Heat Pump Package Unit(CU-02),CU-02,MITSUBISHI,PUZ-HA30NHA5,83U03838B
508864,DXC Technologies,US,LUS105393,dc17afcb-a7c3-4ed7-bea0-3419702fc704,AUS123524,ac028380-f423-492e-90e7-52d75253d1e9,AUS123524,c732e572-361a-427d-bc80-c16517fe0118,US-MGG-DC-738911- FCU003,US-MGG-DC-738911- FCU003,CARRIER,30RBP-200,65638-3
508865,DXC Technologies,US,LUS103385,f0bcb372-041e-44bd-8c21-cd97ba508c0c,AUS131260,cef54ee4-2d2a-4694-8e99-4b99b3957fc7,AUS131260,5a5f1889-5117-4764-8214-79cea7cd3e0b,US-NOR-DC-5513645 AHU4 SUPPLY FAN ROOF,US-NOR-DC-5513645 AHU4 SUPPLY FAN ROOF,CARRIER,30GTN210-K620KA,2405F16912-1
